<div style="border: 2px solid #00008B; padding: 15px; border-radius: 10px; background-color: #00008B; color: #FFFFFF; font-family: Arial;">
  <h1 style="margin-top: 0;">Prática: Reconhecimento de Entidades Nomeadas (Named Entity Recognition - NER)</h1>
</div>

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
<b>Descrição: Encontrar as entidades nomeadas no processo</b> </br></br> 

Número do processo: 123456-12.3456.7.89.1234</br>
Classe judicial: PROCEDIMENTO COMUM CÍVEL</br>
REQUERENTE: MINHA EMPRESA ADVOGADOS ASSOCIADOS S/S</br>
REQUERIDO: PEDRO ALVARES CABRAL</br>

DECISÃO INTERLOCUTÓRIA</br>
(Emenda à Inicial)</br>

Convido o autor a promover a emenda à inicial, no prazo de 15 dias, a fim cumprir as disposições constantes dos itens abaixo, sob pena de incidência do art. 321, parágrafo único, do Código de Processo Civil:
1) Juntar procuração atualizada, haja vista que o instrumento de ID 123456789 está datado do ano de 2022.
2)Esclarecer o ajuizamento da ação na jurisdição do Brasília/DF, haja vista que no contrato objeto da ação (ID 123456789, cláusula VIII, item 24), consta que as partes elegeram o foro da cidade de Goiânia/Goiás. Com efeito, após recente alteração, o §1º do art. 63 do CPC passou a dispor que "a eleição de foro somente produz efeito quando constar de instrumento escrito, aludir expressamente a determinado negócio jurídico e guardar pertinência com o domicílio ou a residência de uma das partes ou com o local da obrigação, ressalvada a pactuação consumerista, quando favorável ao consumidor". No caso dos autos, o foro eleito pelas partes guarda pertinência com o domicílio do requerente, ID 123456789.
3) Comprovar a efetiva atuação do causídico, ora requerente, na demanda objeto do contrato entabulado entre as partes. Para tanto, junte-se cópia integral do referido processo, até o momento em que efetivada a renúncia.
Sem prejuízo, à Secretaria para retificação da autuação, alterando a classe processual para "Execução de Título Extrajudicial".

Intime-se.

documento assinado digitalmente</br>
PERO VAZ DE CAMINHA</br>
Juíz de Direito</br>

</div>

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
    
O **Reconhecimento de Entidades Nomeadas (Named Entity Recognition - NER)** é uma sub-tarefa da extração de informações que busca <span style="color:red">localizar e classificar entidades nomeadas mencionadas em texto não estruturado em categorias predefinidas, como nomes de pessoas, organizações, locais, códigos médicos, expressões de tempo, quantidades, valores monetários, porcentagens</span>, etc.

O Reconhecimento de Entidades Nomeadas (NER) é um processo onde um algoritmo recebe uma cadeia de texto (sentença ou parágrafo) como entrada e identifica substantivos relevantes (pessoas, lugares e organizações) mencionados nesse texto. Isso pode ajudar em muitas aplicações comerciais, como organizar grandes conjuntos de dados de avaliações de clientes ou até mesmo em tarefas de extração de informações, como a construção de um grafo de conhecimento.

</div>

<h1><span style="color:red">Neste exemplo vamos usar o GOOGLE COLAB.</span></h1>

Faça Upload desse Notebook no https://colab.research.google.com/

# Utilizando o Llama 3 do Grog

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
    
**Groq**, a startup baseada em Mountain View, Califórnia, que <span style="color:red">produz microchips projetados especificamente para executar grandes modelos de linguagem (LLMs) de forma rápida e eficiente </span>. O **GroqCloud é um playground que permite que os desenvolvedores** de IA acessem o Groq Language Processing Unit (LPU) Inference Engine.

</div>


### Passo 1: Crie sua conta e API Key no Groq

https://console.groq.com/playground

Para usar o LPU e os modelos diretamente no código você vai precisar criar uma **API Key no Groq**.

*   Visite o site da Groq Cloud em console.groq.com e crie uma conta.
*   Faça Login na sua conta e navegue até a seção de API Keys e gere uma Nova Chave de API

### Passo 2: Configure a API Key no Google Colab

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
  
Para poder usar a **API Key no Google Colab**:

*   Abra o Google Colab e vá para Secrets.
*   Digite o Nome e o Valor do segredo. Embora o Valor possa ser alterado, o Nome não pode ser alterado. Neste código o Nome é **GROQ**, mas você pode alterar
*   Ative o acesso ao Notebook.
*   Finalmente, para usá-lo no notebook, use o código fornecido com o nome do seu segredo no lugar de \<secretName\>

https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75

</div>


In [ ]:
!pip install -q groq

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

import time
import re
import gc

from google.colab import userdata, drive, files

from groq import Groq
from tqdm import tqdm

In [ ]:
# Instancia Groq Client
gclient = Groq(
    api_key=userdata.get("GROQ"),
)

# Montando o Drive

### Passo 3: No seu Drive do Google crie a pasta LlamaModels

Vamos aportar os arquivos e modelo para serem armazenados nessa pasta. É necessário ter espaço no Drive para isso

In [ ]:
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/LlamaModels'
model_path = '/content/drive/MyDrive/LlamaModels'

# Carga de dados

### Passo 4: Faça Upload do arquivo decisao.txt na pasta LlamaModels do Drive


In [ ]:
with open("/content/drive/MyDrive/LlamaModels/decisao.txt", "r") as file1:
    dataset = file1.read()
print(dataset)

# Tratamento dos dados

In [ ]:
# Criar um padrão regex que corresponde a qualquer um dos delimitadores
def split_text(text, delimiters):
    regex_pattern = '|'.join(map(re.escape, delimiters))
    return re.split(regex_pattern, text)

delimiters = ['\n', '.', ',']
#delimiters = ['\n', '.', ':']

partes = split_text(dataset, delimiters)

# Remover espaços extras e adicionar à lista de resultados se não estiver vazia
contexto = [parte.strip() for parte in partes if parte.strip()]

print(contexto)

# Defina Prompt

<div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px;">
Você precisa formatar a entrada para um dos formatos:
    
*    **conversational format**
      *    {"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}


*    **instruction format**
      *    {"prompt": "<prompt text>", "completion": "<ideal generated text>"}
</div>

In [ ]:
# Mensagem do Sistema
system_message = """Você é um experiente analista judiciário.
Extraia todas as entidades nomeadas e os relacionamento entre elas do contexto.
Escreva a resposta como uma tupla entidade 1|relacionamento|entidade 2.
Não adicione mais nada. Use apenas o contexto do usuário. Caso não encontre entidades escreve "Não existe entidades no contexto"
Exemplo de texto: João trabalha no TJDFT.
Resposta: João|trabalha|TJDFT.
"""

messages = [[
    {"role": "system","content": f"{system_message}"},
    {"role": "user", "content": input}] for input in contexto]

In [ ]:
messages[10]

# Gera uma resposta usando a função chat completion

In [ ]:
# Label do Llama3 no GroqCloud
model="llama3-70b-8192"

def process_data_groq(prompt):
    chat_completion = gclient.chat.completions.create(
        messages=prompt,
        model=model,
        temperature=0.5,
        max_tokens=128,
        top_p=1,
        stop=None,
        stream=False,
    )
    return chat_completion.choices[0].message.content

# Envia mensagens em lotes (batches) para processamento e coleta as respostas

In [ ]:
def send_messages(messages):
    answers=[]
    batch_size=10

    #tqdm é usado para mostrar uma barra de progresso
    for i in tqdm(range(0, len(messages), batch_size)):
        batch = messages[i:i+10]
        print(batch)

        for message in batch:
            output = process_data_groq(message)
            answers.append(output)

        if i +10 < len(messages):
            time.sleep(5)
    return answers

In [ ]:
# Gerar os dados
answers = send_messages(messages)

In [ ]:
answers

In [ ]:
#Monta um dicionário com as respostas
answers = [{'Contexto': user, 'Resposta': output} for user, output in zip(contexto, answers)]
df = pd.DataFrame(answers)
df